# Módulo 4 - Jupyter Avançado e 
#   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Reprodutibilidade
## Aula 3 - Visualizações Ricas

Até o momento, apresentamos como usar visualizações existentes de bibliotecas e como usar visualizações em mágicas e funções.

Este notebook apresenta como estender a visualização no IPython para outras classes.

Vamos começar carregando a extensão

In [1]:
%load_ext spotify

E os dados de música e artista:

In [2]:
import pandas as pd
dfc = pd.read_csv("../dataset/spotify_charts_complete.tsv", sep="\t")
dfa = pd.read_csv("../dataset/spotify_artists_info_complete.tsv", sep="\t")

Como esperado, conseguimos usar a extensão aqui para exibir um tocador de música:

In [3]:
%track {dfc.loc[0,"song_id"]} -h 80

Vamos criar uma classe que representa uma linha do arquivo de artistas:

In [4]:
class Artist:
    def __init__(self, row):
        self.row = row
    def __repr__(self):
        return str(self.row['name'])
    def _repr_html_(self):
        html = %artist {self.row.artist_id}
        return html.data
    def _repr_markdown_(self):
        return f"# {self.row['name']}"
artist = Artist(dfa.loc[0])

Nessa classe, definimos diversas formas de representar o artista. 

A representação `__repr__` é a padrão do Python para texto.

A representação `_repr_html_` é a representação em HTML do objeto. Para essa representação, estamos usando a mágica `%artist`.

A representação `_repr_markdown_` é a representação em Markdown do objeto. Para essa representação, estamos pegando o nome do artista e usando como título do markdown.

Além desses métodos, é possível especificar:

- `_repr_svg_`
- `_repr_png_`
- `_repr_jpeg_`
- `_repr_javascript_`
- `_repr_latex_`

Se quisermos visualizar o artista com a representação Markdown podemos especificar na função `display` o Mimetype desejados:

In [5]:
display(artist, include=["text/markdown"])

# Coldplay

Se não especificarmos nada, o notebook seguirá a seguinte ordem de prioridade por padrão:

- HTML
- Markdown
- LaTeX
- SVG
- PNG
- JPEG
- Javascript
- Texto


Mas é possível que a ordem mude de acordo com a aplicação.

Jupyter console não exibe HTML nem Javascript, por exemplo.

Essa ordem de exibição também vale para a representação de objetos no notebooks. Portanto, se uma instância de `artist` for a última expressão de uma célula, ela será exibida como HTML:

In [6]:
display(artist)

Coldplay

## Alternativa
Uma alternativa a definição de um método `_repr_<tipo>_` para cada tipo, é a definição de um único método `_repr_mimebundle_` para retornar todas as exibições suportadas.

A seguir usamos `_repr_mimebundle_` para definir visualizações da classe `Track`.

In [7]:
class Track:
    def __init__(self, row):
        self.row = row
    def _repr_mimebundle_(self, **kwargs):
        row = self.row
        html = %track {row.song_id}
        return {
          'text/markdown': f"{row.artist} - "
                           f"*{row.track_name}*",
          'text/html': html.data,
          'trk.spotify+json': row.to_json(),
          'text/plain': row.track_name 
        }
track = Track(dfc.loc[0])

A função `display` pode ser usada da mesma forma para exibir cada tipo.

In [8]:
display(track, include=["text/markdown"])

Tones And I - *Dance Monkey*

In [9]:
display(track, include=["text/plain"])

Dance Monkey

In [10]:
track

Dance Monkey

Note que é possível exibir também tipos que a princípio não são suportados.

In [11]:
display(track, include=["trk.spotify+json"])

Esses tipos podem ser suportados através de extensões na interface do Jupyter (labextension, nbextension) e em geral são registrados com prioridade máxima na exibição.

Mas por questão de tempo e escopo, não mostraremos como fazer neste minicurso.

## Conclusão

Este notebook apresentou duas formas de definir visualizações ricas para objetos do Python.

O próximo notebook ([4.4.Widget.ipynb](4.4.Widget.ipynb)) apresenta como criar um widget interativo simples.